<a href="https://colab.research.google.com/github/suarez-oa/web_scraping/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import date
import csv

In [ ]:
url = 'https://cuspide.com/100-mas-vendidos/'
r = requests.get(url)

if r.status_code == 200:
    # El código 200 indica que la solicitud fue exitosa
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
else:
    print("Error en la solicitud web. Código de estado:", r.status_code)



links = soup.find_all('a')
enlaces = []
nombres = []
urls_imagenes = []
for link in soup.find_all('a', href=True):
    href = link.get('href')
    if '/producto/' in href:
        enlaces.append(href)
        nombres.append(link.text.strip())
        img_tag = link.find('img')
        if img_tag and img_tag.has_attr('src'):
            url_imagen = img_tag['src']
            urls_imagenes.append(url_imagen)
        else:
            urls_imagenes.append(None)


enlaces_sd = []
for j in enlaces:
    if j not in enlaces_sd:
        enlaces_sd.append(j)


nombres = [elemento.replace(".", "").strip() for elemento in nombres]
nombres = [elemento for elemento in nombres if elemento]



bdi_elements = soup.find_all('bdi')
precio_pesos = []

for bdi_element in bdi_elements:
    precio = bdi_element.text.strip()
    precio_pesos.append(precio)


precio_pesos = [elemento.replace('$', '').replace('.', '').replace(',', '.').strip() for elemento in precio_pesos]
precio_pesos_array = np.array(precio_pesos, dtype=float)


precio_pesos_array = np.delete(precio_pesos_array, 0)



precio_dolares = []

for url_libro in enlaces_sd:
    try:
        precio_response = requests.get(url_libro)
        if precio_response.status_code == 200:
            html_precio = precio_response.text
            precio_bus = BeautifulSoup(html_precio, features='html.parser')
            precio_libro_usd = precio_bus.find('span', style='font-size: 1.3em')

            if precio_libro_usd:
                precio_dolares.append(precio_libro_usd.text.strip().replace(',','.'))
            else:
                print(f'No se encontró el precio en la página: {url_libro}')
        else:
            print(f'Error en la solicitud de la página: {url_libro}')
    except Exception as e:
        print(f'Error al procesar el enlace {url_libro}: {str(e)}')



url = 'https://www.cronista.com/MercadosOnline/dolar.html'
r = requests.get(url)

if r.status_code == 200:
    # El código 200 indica que la solicitud fue exitosa
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

else:
    print("Error en la solicitud web. Código de estado:", r.status_code)


div_sell_values = soup.find_all('div', class_='sell-value')


precios = []

for div in div_sell_values[:2]:
    span = div.find('span', class_='currency')
    precio = div.text.strip()


    precios.append(precio)

precios = [e.replace('$', '').replace(',', '.').strip() for e in precios]
precios_array = np.array(precios, dtype=float)


dolar_blue = precios_array[1]

precio_dolar_blue = np.round(precio_pesos_array / dolar_blue, decimals=2)

print('Dolar Blue:', dolar_blue)


fecha = date.today()

lengths = [len(nombres), len(enlaces_sd), len(precio_pesos_array), len(precio_dolares), len(precio_dolar_blue)]
max_length = max(lengths)

lengths = [len(nombres), len(enlaces_sd), len(precio_pesos_array), len(precio_dolares), len(precio_dolar_blue)]
max_length = max(lengths)


nombres = nombres[:max_length]
enlaces_sd = enlaces_sd[:max_length]
urls_imagenes = urls_imagenes[:max_length]
precio_pesos_array = precio_pesos_array[:max_length]
precio_dolares = precio_dolares + [None] * (max_length - len(precio_dolares))  # Añadir None para precios faltantes
precio_dolar_blue = precio_dolar_blue[:max_length]

# Crear el DataFrame
datos = {'titulo': nombres, 'url': enlaces_sd, 'url_img': urls_imagenes, 'precio': precio_pesos_array, 'precio_usd': precio_dolares, 'precio_usd_blue': precio_dolar_blue, 'fecha': fecha}
df = pd.DataFrame(datos)

# Guardar en CSV
df.to_csv('cuspide_OA.csv', index=False, encoding='utf-8')

No se encontró el precio en la página: https://cuspide.com/producto/zensorialmente/
No se encontró el precio en la página: https://cuspide.com/producto/habitos-atomicos/
No se encontró el precio en la página: https://cuspide.com/producto/lo-mejor-y-lo-peor-de-internet/
No se encontró el precio en la página: https://cuspide.com/producto/la-sociedad-de-la-nieve/
No se encontró el precio en la página: https://cuspide.com/producto/deja-de-ser-tu/
No se encontró el precio en la página: https://cuspide.com/producto/como-hacer-que-te-pasen-cosas-buenas/
No se encontró el precio en la página: https://cuspide.com/producto/nuestra-parte-de-noche/
No se encontró el precio en la página: https://cuspide.com/producto/cuando-no-queden-mas-estrellas-que-contar/
No se encontró el precio en la página: https://cuspide.com/producto/el-camino-del-artista/
No se encontró el precio en la página: https://cuspide.com/producto/la-paciente-silenciosa/
No se encontró el precio en la página: https://cuspide.com/pr

ValueError: could not convert string to float: '1.220.00'